In [ ]:
%pip install -U "unstructured[all-docs]" pillow lxml pillow
%pip install -U chromadb tiktoken
%pip install -U langchain langchain-community langchain-openai langchain-groq
%pip install -U python_dotenv

In [ ]:
%pip install -U langchain-openrouter

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env file

Extract the data

Extract the elements of the PDF that we will be able to use in the retrieval process. These elements can be: Text, Images, Tables, etc.

In [ ]:
from unstructured.partition.pdf import partition_pdf

output_path = "./content/"
file_path = output_path + 'attention.pdf'


chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables

    extract_image_block_types=["Image", "Table"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage

    chunking_strategy="by_title",          # or 'basic'
    max_characters=10000,                  # defaults to 500
    combine_text_under_n_chars=2000,       # defaults to 0
    new_after_n_chars=6000,

    
)

In [ ]:
set([str(type(el)) for el in chunks])

In [ ]:
len(chunks)

In [ ]:
chunks[70].metadata.orig_elements

In [ ]:
elements = chunks[3].metadata.orig_elements
chunk_images = [el for el in elements if "Image" in str(type(el))]
chunk_images[0].to_dict()

In [10]:
# counting tables in all chunks
chunk_tables = []
for i in range(len(chunks)):
    elements = chunks[i].metadata.orig_elements
    chunk_tables.extend([el for el in elements if "Table" in str(type(el))])
print(len(chunk_tables))

4


In [ ]:
import base64
from IPython.display import Image, display

def display_base64_image(base64_code):
    # Decode the base64 string to binary
    image_data = base64.b64decode(base64_code)
    # Display the image
    display(Image(data=image_data))

display_base64_image(images[0])


In [10]:
# Separate extracted elements into tables, text, and images

In [13]:
# separate tables from texts
tables = []
texts = []

for chunk in chunks:
    elements = chunk.metadata.orig_elements
    for element in elements:
        if "Table" in str(type(element)):
            tables.append(element)

    if "CompositeElement" in str(type((chunk))):
        texts.append(chunk)

In [ ]:
tables

In [17]:
# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks)

In [ ]:
%pip install -Uq langchain-groq

In [18]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [19]:

# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain

model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")


summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [21]:
import time
from groq import RateLimitError


try:
    # Summarize text
    text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})

    # Summarize tables
    tables_html = [table.metadata.text_as_html for table in tables]
    table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 1})
except RateLimitError as e:
    
    print(f"Rate limit exceeded: {e}")
    time.sleep(300)
    # Retry summarization
    # Summarize text
    text_summaries = summarize_chain.batch(texts, {"max_concurrency": 1})

    # Summarize tables
    tables_html = [table.metadata.text_as_html for table in tables]
    table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 1})

In [ ]:
print(len(text_summaries))
print(len(table_summaries))

In [ ]:
%pip install -Uq langchain_openai

In [ ]:
import os, re, time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.api_core.exceptions import ResourceExhausted

# Make sure you have your API key set:
# os.environ["GOOGLE_API_KEY"] = "your_google_ai_studio_key"

api_key = os.getenv("GOOGLE_STUDIO_API_KEY")

# Define prompt
prompt_template = """Describe the image in detail. For context,
                  the image is part of the research paper by google Attention Is All You Need
Guide book. Be specific about graphs, such as bar plots."""

# Template for multimodal message (same as before)
messages = [
    (
        "user",
        [
            {"type": "text", "text": prompt_template},
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image}"},
            },
        ],
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

# Use Gemini 2.0 Flash-Lite (best rate & token limits)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.5,
    google_api_key=api_key,
)

chain = prompt | llm | StrOutputParser()

# Assuming 'images' is a list of base64-encoded images
while True:
    try:
        image_summaries = chain.batch(images)
        break  # success, exit loop
    except ResourceExhausted as e:
        msg = str(e)
        # Detect rate limit and extract wait time if possible
        match = re.search(r'Please try again in ([\d\.]+)([sm])', msg)
        wait_seconds = 60  # default wait time
        if match:
            value, unit = match.groups()
            wait_seconds = float(value) * 60 if unit == 'm' else float(value)
        print(f"⏳ Rate limit exceeded. Waiting {wait_seconds:.2f} seconds...")
        time.sleep(wait_seconds)
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")
        time.sleep(5)


In [ ]:
print(image_summaries[0])

In [ ]:
%pip install langchain-huggingface

In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install -Uq InstructorEmbedding

In [50]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from sentence_transformers import SentenceTransformer


embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

# The vectorstore to index child chunks
vectorstore = Chroma(
    collection_name="multi_modal_rag",
    embedding_function=embeddings
)

# The storage layer for parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)


In [51]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=summary, metadata={id_key: table_ids[i]}) for i, summary in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in images]
summary_img = [
    Document(page_content=summary, metadata={id_key: img_ids[i]}) for i, summary in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, images)))

In [ ]:
print("Total documents in vectorstore:", vectorstore._collection.count())


In [ ]:
docs = vectorstore.get(include=["metadatas", "documents"], limit=5)
for i, (doc, meta) in enumerate(zip(docs["documents"], docs["metadatas"])):
    print(f"\n--- Document {i+1} ---")
    print(f"Metadata: {meta}")
    print(f"Content: {doc[:300]}...")  # Print first 300 chars only


In [ ]:
sample_embedding = vectorstore._collection.get(limit=1, include=["embeddings"])["embeddings"][0]
print("Embedding length:", len(sample_embedding))


In [ ]:
results = vectorstore.similarity_search("self attention", k=3)
for doc in results:
    print("\n🔹 Retrieved:", doc.page_content[:300], "...")
    print("Metadata:", doc.metadata)


In [ ]:
docs = vectorstore.get(include=["metadatas"])
types = [d.get("type", "unknown") for d in docs["metadatas"]]
from collections import Counter
print(Counter(types))


RAG Pipeline

In [65]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
from base64 import b64decode
import os


# --- Helper to separate base64 images from text docs ---
def parse_docs(docs):
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception:
            text.append(doc)
    return {"images": b64, "texts": text}


# --- Build the final multimodal prompt ---
def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            if hasattr(text_element, "page_content"):
                context_text += text_element.page_content + "\n"
            else:
                context_text += str(text_element) + "\n"

    prompt_template = f"""
    You are an expert assistant. Answer the user's question **only** based on the given context.

    Context:
    {context_text}

    Question:
    {user_question}

    Provide a clear and concise answer.
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    # attach images if any
    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"}
            })

    return ChatPromptTemplate.from_messages([
        HumanMessage(content=prompt_content),
    ])


# --- Build the chain ---
chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatOpenAI(
        model="qwen/qwen2.5-vl-32b-instruct:free",  # Qwen model on OpenRouter
        base_url="https://openrouter.ai/api/v1",
        api_key=os.getenv("OPENROUTER_API_KEY"),
        temperature=0.2,
        max_tokens=800
    )
    | StrOutputParser()
)

# --- Optional: RAG with source output ---
chain_with_sources = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough().assign(
        response=(
            RunnableLambda(build_prompt)
            | ChatOpenAI(
                model="qwen/qwen2.5-vl-32b-instruct:free",
                base_url="https://openrouter.ai/api/v1",
                api_key=os.getenv("OPENROUTER_API_KEY"),
                temperature=0.2,
                max_tokens=800
            )
            | StrOutputParser()
        )
    )
)


In [ ]:
response = chain_with_sources.invoke("What is the attention mechanism? can you show me an image fo it?")
print("Response:", response['response'])

print("\n\nContext:")
for text in response['context']['texts']:
    print(text.text)
    print("Page number: ", text.metadata.page_number)
    print("\n" + "-"*50 + "\n")
for image in response['context']['images']:
    display_base64_image(image)